In [1]:
import os
import numpy as np
import cv2
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def generate_hazy_image(J, depth_map, A, beta=1):
    t = np.exp(-beta * depth_map)
    I = J * t[..., np.newaxis] + A * (1 - t[..., np.newaxis])
    return np.clip(I, 0, 1)

def prepare_training_data(image_path, depth_map_path):
    image = io.imread(image_path)/255.0
    depth_map = io.imread(depth_map_path)/255.0
    depth_map = 1 - depth_map

    A = np.random.uniform(0.7, 1.0)  # 生成隨機大氣光
    # beta = np.random.uniform(0.5, 2.0)  # 隨機選擇霧霾濃度
    hazy_image = generate_hazy_image(image, depth_map, A)

    return image, depth_map, hazy_image, A

def save_params_to_csv(file_name, image_name, A):
    image_name = image_name.replace('.png', '')
    data = pd.DataFrame({
        'image_name': [image_name],
        'A': [A]
    })

    data.to_csv(file_name, mode='a', header=not os.path.exists(file_name), index=False)


def save_images(image, depth_map, hazy_image, img_name, output_dirs):
    io.imsave(os.path.join(output_dirs['image'], img_name), (image * 255).astype(np.uint8))
    io.imsave(os.path.join(output_dirs['depth'], img_name), (depth_map * 255).astype(np.uint8))
    io.imsave(os.path.join(output_dirs['hazy_image'], img_name), (hazy_image * 255).astype(np.uint8))

img_folder = r"C:\Users\user\Desktop\Xiao\2024.08.17\generate hazy image\white\mix_white_landscape_A\found_white_landscape 1207"
depth_folder = r"C:\Users\user\Desktop\Xiao\pexels-110k-512p-min-jpg-depth\images"

output_dirs = {
    'image': 'image',
    'depth': 'depth',
    'hazy_image': 'hazy_image'
}

# 確保輸出資料夾存在
for dir_path in output_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

img_images = []
depth_images = []
img_names = []
depth_names = []

img_count = 0
max_images = 1300

# 遍歷img資料夾中的文件
for img_filename in os.listdir(img_folder):
    if img_filename.endswith('.png'):
        # img_base_name = img_filename[-11:-4]
        img_base_name = img_filename.split('-')[-1].replace('.png', '')

        for depth_filename in os.listdir(depth_folder):
            # if depth_filename.endswith('.jpeg') and depth_filename[-12:-5] == img_base_name:
            if depth_filename.endswith('.jpeg') and depth_filename.split('-')[-1].replace('.jpeg', '') == img_base_name:
                depth_image_path = os.path.join(depth_folder, depth_filename)

                if os.path.exists(depth_image_path):
                    img_image_path = os.path.join(img_folder, img_filename)
                    img_image = cv2.imread(img_image_path)
                    img_image = cv2.resize(img_image, (224, 224))

                    depth_image = cv2.imread(depth_image_path)
                    depth_image = cv2.cvtColor(depth_image, cv2.COLOR_BGR2GRAY)
                    depth_image = cv2.resize(depth_image, (224, 224))

                    img_name = img_filename.replace('.jpg', '.png')

                    # 準備並保存訓練數據
                    image, depth_map, hazy_image, A = prepare_training_data(img_image_path, depth_image_path)
                    save_images(image, depth_map, hazy_image, img_name, output_dirs)
                    save_params_to_csv('A.csv', img_filename, A )

                    img_images.append(img_image)
                    img_names.append(img_filename)
                    depth_images.append(depth_image)
                    depth_names.append(depth_filename)

                    img_count += 1
                    break

        if img_count >= max_images:
            break

print('Shape of Img:', np.array(img_images).shape)
print('Shape of Depth:', np.array(depth_images).shape)

print(f"讀取了 {len(img_images)} 張 img 圖片")
print(f"讀取了 {len(depth_images)} 張 depth 圖片")

print("Img 圖片名稱列表:", img_names[:10])
print("Depth 圖片名稱列表:", depth_names[:10])


Shape of Img: (1207, 224, 224, 3)
Shape of Depth: (1207, 224, 224)
讀取了 1207 張 img 圖片
讀取了 1207 張 depth 圖片
Img 圖片名稱列表: ['1-person-beauty-face-blurred-background-2586334.png', '10-euros-animal-cat-1383812.png', '100-federal-street-architecture-boston-2980251.png', '102742.png', '103595.png', '1065753.png', '109009.png', '119775.png', '12-apostles-bay-beach-943459.png', '120mm-50mm-landscape-3029705.png']
Depth 圖片名稱列表: ['depth-2586334.jpeg', 'depth-1383812.jpeg', 'depth-2980251.jpeg', 'depth-102742.jpeg', 'depth-103595.jpeg', 'depth-1065753.jpeg', 'depth-109009.jpeg', 'depth-119775.jpeg', 'depth-943459.jpeg', 'depth-3029705.jpeg']
